In [23]:
import numpy as np
from pathlib import Path
from rembg import remove
from PIL import Image
from skimage import feature
import os
from sklearn.cluster import KMeans
from skimage import io

In [24]:
# Cac ham chinh
def acc_his(arr):
    new_arr = []
    s = sum(arr)
    v = 0
    for i in range(len(arr)):
        v += arr[i]
        new_arr.append(v*100/s)
    return new_arr

def changeWhiteBG(url: str):
    path = Path(url)
    in_img = Image.open(path)
    out_img = remove(in_img)
    white_bg = Image.new("RGBA", size=in_img.size, color=(255, 255, 255))
    white_bg.paste(out_img, (0, 0), mask=out_img)
    new_img = white_bg.convert("RGB")
    new_img.save(path)

def getFeature(url: str):
    path = Path(url)
    img = Image.open(path)
    
    # Lay 3 kenh mau r, g, b
    wr = img.getdata(band=0)
    wg = img.getdata(band=1)
    wb = img.getdata(band=2)

    # Xac dinh bien canny
    gray_img = img.convert("L")
    matrix = np.array(gray_img.getdata()).reshape(gray_img.size)
    edge_canny = feature.canny(matrix, low_threshold=10, high_threshold=30)
    height, width = img.size

    # Tinh so pixel nam o khu vuc nen
    co = 0
    for i in range(height):
        if not any(edge_canny[i]):
            co += width
            continue
        limit = 0
        for j in range(width):
            if edge_canny[i][j] == True:
                limit = j
                break
            else:
                co += 1
        for j in range(width-1, limit, -1):
            if edge_canny[i][j] == True:
                # print(i, j)
                break
            else:
                co += 1

    # Xay dung bieu do tan xuat cho 3 kenh mau
    r_his = wr.histogram()
    g_his = wg.histogram()
    b_his = wb.histogram()
    bg_pixel = min(co, r_his[255], g_his[255], b_his[255])
    r_his[255] = r_his[255] - bg_pixel
    g_his[255] = g_his[255] - bg_pixel
    b_his[255] = b_his[255] - bg_pixel

    # Xay dung bo dac trung
    image_feature = []
    image_feature += acc_his(r_his)
    image_feature += acc_his(g_his)
    image_feature += acc_his(b_his)
    # print(len(image_feature))
    return image_feature


In [25]:
import mysql.connector

def getConnection():
    mydb = mysql.connector.connect(
        host="localhost",
        user="root",
        password="Thang*160703",
        database="mmdb_demo"
    )
    return mydb

def insertCentroid(data):
    mydb = getConnection()
    mycursor = mydb.cursor()

    sql = "INSERT INTO centroids ("
    for i in range(256):
        sql += "r_" + str(i) + ", "
    for i in range(256):
        sql += "g_" + str(i) + ", "
    for i in range(255):
        sql += "b_" + str(i) + ", "
    sql += "b_255)"

    sql += " VALUES ("
    for i in range(767):
        sql += "%s, "
    sql += "%s)"
    val = data
    mycursor.executemany(sql, val)
    mydb.commit()
    print(mycursor.rowcount, "inserted.")
    mydb.close()

def insertFeatures(data):
    mydb = getConnection()
    mycursor = mydb.cursor()

    sql = "INSERT INTO features ("
    sql += "centroid_id, "
    for i in range(256):
        sql += "r_" + str(i) + ", "
    for i in range(256):
        sql += "g_" + str(i) + ", "
    for i in range(256):
        sql += "b_" + str(i) + ", "
    sql += "link)"

    sql += " VALUES ("
    for i in range(769):
        sql += "%s, "
    sql += "%s)"
    val = data
    mycursor.executemany(sql, val)
    mydb.commit()
    print(mycursor.rowcount, "inserted.")
    mydb.close()

def getAllData(table: str):
    mydb = getConnection()
    mycursor = mydb.cursor()

    mycursor.execute("SELECT * FROM " + table)
    myresult = mycursor.fetchall()
    mydb.close()

    return myresult

def deleteAll(table: str):
    mydb = getConnection()
    mycursor = mydb.cursor()
    sql = "DELETE FROM " + table
    mycursor.execute(sql)
    mydb.commit()
    print(mycursor.rowcount, "record(s) deleted")
    mydb.close()

def updateFeatures(data):
    mydb = getConnection()
    mycursor = mydb.cursor()

    sql = "UPDATE features SET centroid_id = %s WHERE id = %s"
    val = (data[1], data[0])
    mycursor.execute(sql, val)
    mydb.commit()
    mydb.close()

def getFeaturesData(centroid_id: int):
    mydb = getConnection()
    mycursor = mydb.cursor()

    sql = "SELECT * FROM features WHERE centroid_id = " +  str(centroid_id)
    mycursor.execute(sql)
    myresult = mycursor.fetchall()
    mydb.close()
    return myresult

In [22]:
# Chuyen ve nen trang
# path = 'D:\\Projects\\Multimedia-DB\\photo-v2\\thit-bo-bit-tet\\'
# for x in os.listdir(path):
#     if x.endswith(".jpg"):
#         changeWhiteBG(url=path + x)

In [26]:
# Xac dinh bo dac trung
path = 'D:\\Projects\\Multimedia-DB\\photo-MMDB\\'
data = []
for x in os.listdir(path):
    folderPath = path + x + "\\"
    for y in os.listdir(path=folderPath):
        if y.endswith(".jpg"):
            newPath = folderPath + y
            imageFeature = [None] + getFeature(url=newPath)
            imageFeature.append(newPath)
            data.append(imageFeature)

In [29]:
# Xac dinh tam cac cum va luu vao CSDL
centroidNum = 10
train = []
for i in data:
    train.append(i[1:769])

built_in_kmean = KMeans(n_clusters=centroidNum, random_state=0, tol=0.0001).fit(train)
clusterCenters = built_in_kmean.cluster_centers_
centroids = []

for i in clusterCenters:
    centroids.append(tuple(i))

deleteAll(table="centroids")
insertCentroid(data=centroids)

4 record(s) deleted
10 inserted.


In [30]:
def square_distance(a: list, b: list):
    val = 0
    for i in range(len(a)):
        val += (a[i] - b[i])**2
    return val

def findCentroid(centroids):
    min_distance = -1
    center = -1
    for i in centroids.keys():
        if (min_distance < 0) or (min_distance > centroids[i]):
            min_distance = centroids[i]
            center = i
    centroids.pop(center)
    return center

def showInTerminal(path):
    img = io.imread(Path(path))
    io.imshow(img)

def findSimilarImg(imgFeature):
    savedCentroids = getAllData(table="centroids")
    disDict = {}
    for i in savedCentroids:
        disDict[i[0]] = square_distance(i[1:], imgFeature)
    count = 10

    featureDict = {}
    distance = []
    while len(featureDict) < count:
        center = findCentroid(disDict)
        results = getFeaturesData(centroid_id=center)
        for re in results:
            featureDict[re[0]] = re
            distance.append([re[0], square_distance(re[2:770], imgFeature)])

    distance = sorted(distance, key=lambda l:l[1], reverse=False)
    for i in range(count):
        id = distance[i][0]
        # print(id)
        img = featureDict[id][-1]
        print(img)
        # showInTerminal(img)


In [31]:
# Phan cum

savedCentroids = getAllData(table="centroids")
centroidNum = len(savedCentroids)
for i in range(len(savedCentroids)):
    savedCentroids[i] = list(savedCentroids[i])
    # print(savedCentroids[i])

for i in range(len(data)):
    item = data[i]
    imgFeatures = item[1:769]
    centroid = savedCentroids[0]
    item[0] = centroid[0]
    minDis = square_distance(imgFeatures, centroid[1:])
    for j in range(1, centroidNum):
        centroid = savedCentroids[j]
        if minDis > square_distance(imgFeatures, centroid[1:]):
            item[0] = centroid[0]

for i in range(len(data)):
    data[i] = tuple(data[i])
    # print(data[i])

In [32]:
print(data[0])

(19, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00026276513001618633, 0.000788295390048559, 0.0018393559101133044, 0.003153181560194236, 0.006306363120388472, 0.013401021630825504, 0.02575098274158626, 0.04335624645267074, 0.07699018309474259, 0.11614218746715436, 0.17657816737087723, 0.2364886170145677, 0.31216497445922936, 0.3841626200836644, 0.4942612095604465, 0.5793971116856909, 0.7081520253936222, 0.8224548569506632, 0.9425385213680604, 1.0794391541064934, 1.2194929684051208, 1.3797796977149945, 1.516154800193395, 1.645960774421391, 1.8133421622417019, 1.957337453490572, 2.0821508902482604, 2.2384961426078913, 2.4274242710895293, 2.6029513779403417, 2.75535515334973, 2.917481238569717, 3.0785562632696393, 3.2154568960080723, 3.349467112316327, 3.48636774505476, 3.657690609825314, 3.8248092325156082, 4.000336339366421, 4.203191019738917, 4.3939585041306675, 4.587616404952597, 4.774705177524122, 4.959166298795485, 5.127861512265876, 5.30811839145698, 5.462098757646466

In [33]:
# Luu thong tin dac trung vao CSDL
# deleteAll(table="features")
# insertFeatures(data=data)

44 record(s) deleted
110 inserted.


In [62]:
path = "D:\Projects\Multimedia-DB\photo-test\\xuc-xich.jpg"
path = path.replace("\\", "\\\\")

try:
    Image.open(path)
except:
    print("Co loi xay ra.")
else:
    imgFeature = getFeature(url=path)
    # print(imgFeature)
    # showInTerminal(path=path)
    findSimilarImg(imgFeature=imgFeature)


D:\Projects\Multimedia-DB\photo-MMDB\ga-luoc\5.jpg
D:\Projects\Multimedia-DB\photo-MMDB\ga-luoc\1.jpg
D:\Projects\Multimedia-DB\photo-MMDB\ga-luoc\4.jpg
D:\Projects\Multimedia-DB\photo-MMDB\mi-y\6.jpg
D:\Projects\Multimedia-DB\photo-MMDB\ga-luoc\7.jpg
D:\Projects\Multimedia-DB\photo-MMDB\mi-y\8.jpg
D:\Projects\Multimedia-DB\photo-MMDB\sushi-ca-hoi\7.jpg
D:\Projects\Multimedia-DB\photo-MMDB\ga-luoc\9.jpg
D:\Projects\Multimedia-DB\photo-MMDB\ga-luoc\8.jpg
D:\Projects\Multimedia-DB\photo-MMDB\sushi-ca-hoi\3.jpg


In [44]:
# deleteAll(table="centroid")
# deleteAll(table="features")